In [93]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.activations import sigmoid
np.random.seed(1)
from model_helper import load_test_case, divide_data
from model_helper import corr_plt
import os
import pandas as pd
import datetime as dt
import numpy as np


def custom_activation(x):
    return (sigmoid(x)/5) - 0.1

def all_stock_name():
    filenames = os.listdir(path)
    temp = [filename for filename in filenames if filename.endswith('.csv')]
    #print temp
    return [x.replace('.csv' ,'') for x in temp]


def single_stock_data(stock_name):
    data_path = os.path.join(path,stock_name+'.csv')
    df = pd.DataFrame.from_csv(data_path,header = None, parse_dates=[[0, 1]])
    df.columns = ['Open','High','Low','Close','Outstanding','Turnover']
    # header=['date','time','open','high','low','close','outstanding','turnover']
    return df


def x_y_new(df):
    N = 60
    M = len(df) - N - 10
    X = np.zeros((M,N,6))
    Y = np.zeros((M,1))
    Z = df.values
    for i in range(M):
        X[i, :, :] = Z[i:i+N,:]
        forward = Z[i+N+1,3] -  Z[i+N,3]
        now = Z[i+N,3] -  Z[i+N-1,3]
        if forward * now < 0:
            Y[i,0] = 1
            
    return X,Y
        
        

def x_y_for_single_stock(df,stock_name):
    print (stock_name)
    all_dates = list(set([x.date() for x in df.index]))
    all_dates.sort()
    N = len(all_dates)-1
    X = np.zeros((N,209,6,1))
    Y = np.zeros((N,1))
    pos = []
    for i in range(N):
        d1=all_dates[i]
        d2=all_dates[i+1]

        x_start,x_end = return_x_time(d1)
        #print x_start, x_end
        x_df = df[x_start:x_end]
        temp_x = x_df.values
        temp_x = np.diff(temp_x,axis=0)
        X[i, :, :, :] = temp_x.reshape((209, 6, 1))

        y_start, y_end = return_y_time(d1)
        y_df = df[y_start:y_end]
        Y[i,0] = y_singal(y_df)
        pos.append([stock_name,d1])

    return X,Y,pos

def return_x_time(date):
    return (dt.datetime(date.year,date.month,date.day,9,0),
            dt.datetime(date.year, date.month, date.day, 14, 30))

def y_singal(df):
    price1 = float(df[df.index == df.index.min()]['Close'])
    price2 = float(df[df.index == df.index.max()]['Close'])
    return float(price2 - price1)/price1

def number_to_category(num):
    return


def return_y_time(d1):
    return (dt.datetime(d1.year,d1.month,d1.day,14,30),
            dt.datetime(d1.year, d1.month, d1.day, 14,31))


path = r"data"
all_names = all_stock_name()
    #print all_names
    
Pos = []

name = all_names[2]
df = single_stock_data(name)
X,Y = x_y_new(df)
    

print(X.shape)
print(Y.shape)

(30170, 60, 6)
(30170, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:30: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


In [89]:
print(X[3,3,:])

[133.747 133.747 133.747 133.747   0.      0.   ]


In [94]:
print(sum(Y))

[462.]


In [95]:
X = X [0:5000,:,:]
Y = Y [0:5000,:]
print(sum(Y))

[76.]


In [96]:
# Model: Price_Forecast
def Price_Forecast(input_shape):
    """
    Function creating the Price_Forecast model's graph.

    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """

    ### START CODE HERE ###
    embeddings = Input(shape=input_shape, dtype=np.float32)
    N =32
    
    print(embeddings.shape, "....")
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(N, return_sequences=True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    
    X = LSTM(N)(embeddings)
    
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(1, activation = 'softmax')(X)

    # Create Model instance which converts embeddings into X.
    model = Model(embeddings, X)

    ### END CODE HERE ###

    return model

def custom_activation(x):
    return (sigmoid(x)/50) - 0.01


In [97]:
from keras.optimizers import adam

train_features, test_features, train_labels, test_labels = divide_data(X, Y)


inputShape = (480, 6)
print(inputShape)
model = Price_Forecast(inputShape)
model.summary()


model.compile(loss='sparse_categorical_crossentropy',
              optimizer=adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), 
              metrics=['accuracy'])

print(train_features.shape)

print(train_labels.shape)

model.fit(train_features,train_labels, epochs=5, batch_size=128, shuffle=True)

    # evaluate the model performance
    #loss, acc = model.evaluate(np.squeeze(test_features, axis=3), test_labels)
    #print("Test accuracy = ", acc)
# test_predict = model.predict(test_features)
# train_predict = model.predict(train_features)

# corr_plt(train_predict,train_labels)
# corr_plt(test_predict,test_labels)

(480, 6)
(?, 480, 6) ....
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 480, 6)            0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 32)                4992      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 33        
Total params: 5,025
Trainable params: 5,025
Non-trainable params: 0
_________________________________________________________________
(4750, 60, 6)
(4750, 1)


ValueError: Error when checking input: expected input_21 to have shape (480, 6) but got array with shape (60, 6)

In [85]:


print(train_predict)

[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
